In [ ]:
# !wget https://raw.githubusercontent.com/mufaddalhamidofficial/tensorflow_course/main/helper_funcs.py
# !wget https://media.githubusercontent.com/media/mufaddalhamidofficial/skimlit_ai/main/data.zip

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from helper_funcs import create_tensorboard_callback, create_checkpoint_callback, plot_loss_curves, compare_historys, unzip_data, calculate_results
import tensorflow_hub as hub
import io

In [ ]:
tf.__version__

In [ ]:
unzip_data("data.zip")
# !rm -rf data.zip

In [ ]:
DIR_NAME_01 = "data/01_percent"
DIR_NAME_1 = "data/1_percent"
DIR_NAME_10 = "data/10_percent"
DIR_NAME_100 = "data/100_percent"

test_df_100 = pd.read_csv(DIR_NAME_100 + "/test.csv")
test_df_100 = test_df_100.iloc[:, 1:]

test_sentences_ = test_df_100.text.to_numpy()
test_labels_ = test_df_100.target.to_numpy()

## Callbacks:

In [ ]:
def get_callbacks(experiment_name, use_early_stopping = True, patience=10, use_tensorboard = True, use_model_checkpoint = True):
    callbacks = [
    ]
    if use_tensorboard:
        callbacks.append(
            create_tensorboard_callback(
                dir_name="skim_lit/tensorboard",
                experiment_name=experiment_name,
            )
        )
    if use_model_checkpoint:
        callbacks.append(
            create_checkpoint_callback(
                dir_name="skim_lit/checkpoint",
                experiment_name=experiment_name,
                monitor="val_accuracy",
            )
        )
    if use_early_stopping:
        
        callbacks.append(tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy",
            patience=patience,
            restore_best_weights=True,
        ))
    return callbacks

# Modelling

## Model m 0: Naive Bayes

### Get and Prepare data

In [ ]:
train_data_10 = pd.read_csv(DIR_NAME_10 + "/train.csv")
train_sentences = train_data_10.text.to_numpy()
train_labels = train_data_10.target.to_numpy()

val_data_10 = pd.read_csv(DIR_NAME_10 + "/val.csv")
val_sentences = val_data_10.text.to_numpy()
val_labels = val_data_10.target.to_numpy()

test_sentences = test_labels_.copy()
test_labels = test_labels_.copy()

### Label encode labels

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels_encoded = np.array(label_encoder.fit_transform(train_labels))
val_labels_encoded = np.array(label_encoder.transform(val_labels))
test_labels_encoded = np.array(label_encoder.transform(test_labels))

train_labels_encoded

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_m_0 = Pipeline(
    [
        ("tfidf", TfidfVectorizer()),
        ("clf", MultinomialNB()),
    ]
)

model_m_0.fit(train_sentences, train_labels_encoded)

model_m_0_val_preds = model_m_0.predict(val_sentences)

model_m_0_val_results = calculate_results(val_labels_encoded, model_m_0_val_preds)
print(model_m_0_val_results)

model_m_0_preds = model_m_0.predict(test_sentences)
model_m_0_results = calculate_results(test_labels_encoded, model_m_0_preds)
print(model_m_0_results)

## Model m Data Preps:

### Get and Prepare data

In [ ]:
train_data_10 = pd.read_csv(DIR_NAME_10 + "/train.csv")
train_sentences = train_data_10.text.to_numpy()
train_labels = train_data_10.target.to_numpy()

val_data_10 = pd.read_csv(DIR_NAME_10 + "/val.csv")
val_sentences = val_data_10.text.to_numpy()
val_labels = val_data_10.target.to_numpy()

test_sentences = test_sentences_.copy()
test_labels = test_labels_.copy()

### One hot encode labels

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse_output=False)
train_labels_one_hot = one_hot_encoder.fit_transform(train_labels.reshape(-1, 1))
val_labels_one_hot = one_hot_encoder.transform(val_labels.reshape(-1, 1))
test_labels_one_hot = one_hot_encoder.transform(test_labels.reshape(-1, 1))

train_labels_one_hot

In [ ]:
class_names = list(one_hot_encoder.categories_[0])
classes_count = len(class_names) # type: ignore
class_names 

In [ ]:
import string

train_chars = [" ".join(list(sentence)) for sentence in train_sentences]
val_chars = [" ".join(list(sentence)) for sentence in val_sentences]
test_chars = [" ".join(list(sentence)) for sentence in test_sentences]

char_lens = [len(sentence.split(' ')) for sentence in train_chars]

seq_char_len = int(np.percentile(char_lens, 90))

alphabet = string.ascii_lowercase + string.digits + string.punctuation

NUM_CHAR_TOKENS = len(alphabet) + 2
seq_char_len, NUM_CHAR_TOKENS, alphabet

## Model m 1

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels_one_hot))
val_dataset = tf.data.Dataset.from_tensor_slices((val_sentences, val_labels_one_hot))
test_dataset = tf.data.Dataset.from_tensor_slices((test_sentences, test_labels_one_hot))

train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=68000,
    output_sequence_length=50,
)

text_vectorizer.adapt(train_sentences)

embedding = tf.keras.layers.Embedding(
    input_dim=len(text_vectorizer.get_vocabulary()),
    output_dim=128,
    mask_zero=True,
)

inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Conv1D(64, 5, activation='relu', padding="same")(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(classes_count, activation="softmax")(x)

model_m_1 = tf.keras.Model(inputs, outputs) # type: ignore

model_m_1.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_1_history = model_m_1.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_1"),
)

model_m_1.load_weights("skim_lit/checkpoint/model_m_1/checkpoint.ckpt")

model_m_1_val_preds = tf.argmax(model_m_1.predict(val_sentences), axis=1)
model_m_1_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_1_val_preds)
print(model_m_1_val_results)

plot_loss_curves(model_m_1_history)

model_m_1_preds = tf.argmax(model_m_1.predict(test_sentences), axis=1)
model_m_1_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_1_preds)
model_m_1_results

## Model m 5

In [ ]:
train_line_numbers_one_hot = tf.one_hot(train_data_10["line_number"].to_numpy(), depth=15)
val_line_numbers_one_hot = tf.one_hot(val_data_10["line_number"].to_numpy(), depth=15)
test_line_numbers_one_hot = tf.one_hot(test_df_100["line_number"].to_numpy(), depth=15)

train_total_lines_one_hot = tf.one_hot(train_data_10["total_lines"].to_numpy(), depth=20)
val_total_lines_one_hot = tf.one_hot(val_data_10["total_lines"].to_numpy(), depth=20)
test_total_lines_one_hot = tf.one_hot(test_df_100["total_lines"].to_numpy(), depth=20)

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences, test_chars, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:

hub_embedding = hub.KerasLayer(
    "https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",
    trainable=False,
    name="universal_sentence_encoder",
)

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=NUM_CHAR_TOKENS,
    output_sequence_length=seq_char_len,
)

char_vectorizer.adapt(train_chars)

char_embedding = tf.keras.layers.Embedding(
    input_dim=len(char_vectorizer.get_vocabulary()),
    output_dim=25,
    mask_zero=True,
)


token_inputs = tf.keras.layers.Input(shape=[], dtype='string', name='token_inputs')
token_embedding = hub_embedding(token_inputs)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_embedding)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(1,), dtype='string', name='char_inputs')
char_vectors = char_vectorizer(char_inputs)
char_embeddings = char_embedding(char_vectors)
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(24))(char_embeddings)
char_model = tf.keras.Model(char_inputs, char_bi_lstm)

line_number_inputs = tf.keras.layers.Input(shape=(15,), dtype=tf.float32, name='line_number_inputs')
line_number_outputs = tf.keras.layers.Dense(32, activation='relu')(line_number_inputs)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(20,), name='total_lines_inputs')
total_lines_outputs = tf.keras.layers.Dense(32, activation='relu')(total_lines_inputs)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)
drop_out = tf.keras.layers.Dropout(0.5)(drop_out)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(final_concatenate)

model_m_5 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_5.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_5_history = model_m_5.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_5"),
)

model_m_5.load_weights("skim_lit/checkpoint/model_m_5/checkpoint.ckpt")

model_m_5_val_preds = tf.argmax(model_m_5.predict(val_dataset), axis=1)
model_m_5_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_5_val_preds)
print(model_m_5_val_results)

plot_loss_curves(model_m_5_history)

model_m_5_preds = tf.argmax(model_m_5.predict(test_dataset), axis=1)
model_m_5_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_5_preds)
model_m_5_results

## Model m 23

In [ ]:
import string

train_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in train_sentences]
val_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in val_sentences]
test_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in test_sentences]

sentence_lengths = [len(sentence.split(' ')) for sentence in train_chars_puntuations]
# sentence_lengths
seq_char_punctuation_len = int(np.percentile(sentence_lengths, 95) / 4) * 4

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    output_sequence_length=seq_char_punctuation_len,
    standardize='lower',
)

char_vectorizer.adapt(train_chars_puntuations)

train_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(train_chars_puntuations)))
val_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(val_chars_puntuations)))
test_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(test_chars_puntuations)))

one_hot_matrix = np.eye(len(char_vectorizer.get_vocabulary()))

train_chars_puntuations_one_hot = one_hot_matrix[train_chars_puntuations_vectorized][:, :, 2:]
val_chars_puntuations_one_hot = one_hot_matrix[val_chars_puntuations_vectorized][:, :, 2:]
test_chars_puntuations_one_hot = one_hot_matrix[test_chars_puntuations_vectorized][:, :, 2:]
train_chars_puntuations_one_hot.shape, val_chars_puntuations_one_hot.shape, test_chars_puntuations_one_hot.shape

In [ ]:
train_line_numbers_one_hot = train_data_10["line_number"].to_numpy()
val_line_numbers_one_hot = val_data_10["line_number"].to_numpy()
test_line_numbers_one_hot = test_df_100["line_number"].to_numpy()

train_total_lines_one_hot = train_data_10["total_lines"].to_numpy()
val_total_lines_one_hot = val_data_10["total_lines"].to_numpy()
test_total_lines_one_hot = test_df_100["total_lines"].to_numpy()

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars_puntuations_one_hot, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars_puntuations_one_hot, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences, test_chars_puntuations_one_hot, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
hub_embedding = hub.KerasLayer(
    "https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",
    trainable=False,
    name="universal_sentence_encoder",
)


token_inputs = tf.keras.layers.Input(shape=[], dtype='string', name='token_inputs')
token_embedding = hub_embedding(token_inputs)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_embedding)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(20,26,), dtype=tf.float32, name='char_puntuation_inputs')
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(24))(char_inputs)
char_outputs = tf.keras.layers.Dense(128, activation='relu')(char_bi_lstm)
char_model = tf.keras.Model(char_inputs, char_outputs)

line_number_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='line_number_inputs')
line_number_outputs = tf.keras.layers.Dense(32, activation='relu')(line_number_inputs)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(1,), name='total_lines_inputs')
total_lines_outputs = tf.keras.layers.Dense(32, activation='relu')(total_lines_inputs)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)
drop_out = tf.keras.layers.Dropout(0.5)(drop_out)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(final_concatenate)

model_m_23 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_23.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_23_history = model_m_23.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_23"),
)

model_m_23.load_weights("skim_lit/checkpoint/model_m_23/checkpoint.ckpt")

model_m_23_val_preds = tf.argmax(model_m_23.predict(val_dataset), axis=1)
model_m_23_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_23_val_preds)
print(model_m_23_val_results)

plot_loss_curves(model_m_23_history)

model_m_23_preds = tf.argmax(model_m_23.predict(test_dataset), axis=1)
model_m_23_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_23_preds)
model_m_23_results

## Model m 25

In [ ]:
import string

train_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in train_sentences]
val_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in val_sentences]
test_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in test_sentences]

sentence_lengths = [len(sentence.split(' ')) for sentence in train_chars_puntuations]
# sentence_lengths
seq_char_punctuation_len = int(np.percentile(sentence_lengths, 95) / 4) * 4

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    output_sequence_length=seq_char_punctuation_len,
    standardize='lower',
)

char_vectorizer.adapt(train_chars_puntuations)

train_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(train_chars_puntuations)))
val_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(val_chars_puntuations)))
test_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(test_chars_puntuations)))

one_hot_matrix = np.eye(len(char_vectorizer.get_vocabulary()))

train_chars_puntuations_one_hot = one_hot_matrix[train_chars_puntuations_vectorized][:, :, 2:]
val_chars_puntuations_one_hot = one_hot_matrix[val_chars_puntuations_vectorized][:, :, 2:]
test_chars_puntuations_one_hot = one_hot_matrix[test_chars_puntuations_vectorized][:, :, 2:]
train_chars_puntuations_one_hot.shape, val_chars_puntuations_one_hot.shape, test_chars_puntuations_one_hot.shape

In [ ]:
def tf_count(t, val):
    elements_equal_to_value = tf.equal(t, val)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    return count
text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=50)
text_vectorizer.adapt(train_sentences)
vocab = text_vectorizer.get_vocabulary()[2:]

words = tf.reshape(tf.strings.split(tf.strings.regex_replace(train_sentences, r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', ""), ' '), shape=[-1])
counts = {word: tf_count(words, word).numpy() for word in vocab}
counts_df = pd.DataFrame(counts.items(), columns=["word", "count"])
# counts_df = counts_df.sort_values("count", ascending=False)
max_tokens = counts_df[counts_df['count'] >= 5].shape[0]
max_tokens

In [ ]:
train_line_numbers_one_hot = train_data_10["line_number"].to_numpy()
val_line_numbers_one_hot = val_data_10["line_number"].to_numpy()
test_line_numbers_one_hot = test_df_100["line_number"].to_numpy()

train_total_lines_one_hot = train_data_10["total_lines"].to_numpy()
val_total_lines_one_hot = val_data_10["total_lines"].to_numpy()
test_total_lines_one_hot = test_df_100["total_lines"].to_numpy()

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars_puntuations_one_hot, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars_puntuations_one_hot, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences, test_chars_puntuations_one_hot, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=50,
)

text_vectorizer.adapt(train_sentences)

embedding = tf.keras.layers.Embedding(
    input_dim=len(text_vectorizer.get_vocabulary()),
    output_dim=1024,
    mask_zero=True,
)

token_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
token_vectorization = text_vectorizer(token_inputs)
token_embedding = embedding(token_vectorization)
token_merge = tf.keras.layers.Flatten()(token_embedding)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_merge)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(20,26,), dtype=tf.float32, name='char_puntuation_inputs')
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(24))(char_inputs)
char_outputs = tf.keras.layers.Dense(128, activation='relu')(char_bi_lstm)
char_model = tf.keras.Model(char_inputs, char_outputs)

line_number_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='line_number_inputs')
line_number_outputs = tf.keras.layers.Dense(32, activation='relu')(line_number_inputs)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(1,), name='total_lines_inputs')
total_lines_outputs = tf.keras.layers.Dense(32, activation='relu')(total_lines_inputs)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)
drop_out = tf.keras.layers.Dropout(0.5)(drop_out)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(final_concatenate)

model_m_25 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_25.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_25_history = model_m_25.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_25"),
)

model_m_25.load_weights("skim_lit/checkpoint/model_m_25/checkpoint.ckpt")

model_m_25_val_preds = tf.argmax(model_m_25.predict(val_dataset), axis=1)
model_m_25_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_25_val_preds)
print(model_m_25_val_results)

plot_loss_curves(model_m_25_history)

model_m_25_preds = tf.argmax(model_m_25.predict(test_dataset), axis=1)
model_m_25_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_25_preds)
model_m_25_results

## Model m 32

In [ ]:
import string

train_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in train_sentences]
val_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in val_sentences]
test_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in test_sentences]

sentence_lengths = [len(sentence.split(' ')) for sentence in train_chars_puntuations]
# sentence_lengths
seq_char_punctuation_len = int(np.percentile(sentence_lengths, 95) / 4) * 4

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    output_sequence_length=seq_char_punctuation_len,
    standardize='lower',
)

char_vectorizer.adapt(train_chars_puntuations)

train_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(train_chars_puntuations)))
val_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(val_chars_puntuations)))
test_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(test_chars_puntuations)))

one_hot_matrix = np.eye(len(char_vectorizer.get_vocabulary()))

train_chars_puntuations_one_hot = one_hot_matrix[train_chars_puntuations_vectorized][:, :, 2:]
val_chars_puntuations_one_hot = one_hot_matrix[val_chars_puntuations_vectorized][:, :, 2:]
test_chars_puntuations_one_hot = one_hot_matrix[test_chars_puntuations_vectorized][:, :, 2:]
train_chars_puntuations_one_hot.shape, val_chars_puntuations_one_hot.shape, test_chars_puntuations_one_hot.shape

In [ ]:
def tf_count(t, val):
    elements_equal_to_value = tf.equal(t, val)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    return count
text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=50)
text_vectorizer.adapt(train_sentences)
vocab = text_vectorizer.get_vocabulary()[2:]

words = tf.reshape(tf.strings.split(tf.strings.regex_replace(train_sentences, r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', ""), ' '), shape=[-1])
counts = {word: tf_count(words, word).numpy() for word in vocab}
counts_df = pd.DataFrame(counts.items(), columns=["word", "count"])
# counts_df = counts_df.sort_values("count", ascending=False)
max_tokens = counts_df[counts_df['count'] >= 5].shape[0]
max_tokens

In [ ]:
train_line_numbers_one_hot = train_data_10["line_number"].to_numpy()
val_line_numbers_one_hot = val_data_10["line_number"].to_numpy()
test_line_numbers_one_hot = test_df_100["line_number"].to_numpy()

train_total_lines_one_hot = train_data_10["total_lines"].to_numpy()
val_total_lines_one_hot = val_data_10["total_lines"].to_numpy()
test_total_lines_one_hot = test_df_100["total_lines"].to_numpy()

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars_puntuations_one_hot, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars_puntuations_one_hot, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences, test_chars_puntuations_one_hot, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=50,
)

text_vectorizer.adapt(train_sentences)

embedding = tf.keras.layers.Embedding(
    input_dim=len(text_vectorizer.get_vocabulary()),
    output_dim=1024,
    mask_zero=True,
)

token_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
token_vectorization = text_vectorizer(token_inputs)
token_embedding = embedding(token_vectorization)
token_merge = tf.keras.layers.Flatten()(token_embedding)
token_x = tf.keras.layers.Dense(512, activation='relu')(token_merge)
token_x = tf.keras.layers.Dense(256, activation='relu')(token_x)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_x)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(20,26,), dtype=tf.float32, name='char_puntuation_inputs')
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(24))(char_inputs)
char_outputs = tf.keras.layers.Dense(128, activation='relu')(char_bi_lstm)
char_model = tf.keras.Model(char_inputs, char_outputs)

line_number_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='line_number_inputs')
line_number_outputs = tf.keras.layers.Dense(32, activation='relu')(line_number_inputs)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(1,), name='total_lines_inputs')
total_lines_outputs = tf.keras.layers.Dense(32, activation='relu')(total_lines_inputs)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)
drop_out = tf.keras.layers.Dropout(0.5)(drop_out)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(final_concatenate)

model_m_32 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_32.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_32_history = model_m_32.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_32"),
)

model_m_32.load_weights("skim_lit/checkpoint/model_m_32/checkpoint.ckpt")

model_m_32_val_preds = tf.argmax(model_m_32.predict(val_dataset), axis=1)
model_m_32_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_32_val_preds)
print(model_m_32_val_results)

plot_loss_curves(model_m_32_history)

model_m_32_preds = tf.argmax(model_m_32.predict(test_dataset), axis=1)
model_m_32_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_32_preds)
model_m_32_results

## Model m 35

In [ ]:
import string

train_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in train_sentences]
val_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in val_sentences]
test_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in test_sentences]

sentence_lengths = [len(sentence.split(' ')) for sentence in train_chars_puntuations]
# sentence_lengths
seq_char_punctuation_len = int(np.percentile(sentence_lengths, 95) / 4) * 4

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    output_sequence_length=seq_char_punctuation_len,
    standardize='lower',
)

char_vectorizer.adapt(train_chars_puntuations)

train_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(train_chars_puntuations)))
val_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(val_chars_puntuations)))
test_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(test_chars_puntuations)))

one_hot_matrix = np.eye(len(char_vectorizer.get_vocabulary()))

train_chars_puntuations_one_hot = one_hot_matrix[train_chars_puntuations_vectorized][:, :, 2:]
val_chars_puntuations_one_hot = one_hot_matrix[val_chars_puntuations_vectorized][:, :, 2:]
test_chars_puntuations_one_hot = one_hot_matrix[test_chars_puntuations_vectorized][:, :, 2:]
train_chars_puntuations_one_hot.shape, val_chars_puntuations_one_hot.shape, test_chars_puntuations_one_hot.shape

In [ ]:
train_line_numbers_one_hot = train_data_10["line_number"].to_numpy()
val_line_numbers_one_hot = val_data_10["line_number"].to_numpy()
test_line_numbers_one_hot = test_df_100["line_number"].to_numpy()

train_total_lines_one_hot = train_data_10["total_lines"].to_numpy()
val_total_lines_one_hot = val_data_10["total_lines"].to_numpy()
test_total_lines_one_hot = test_df_100["total_lines"].to_numpy()

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars_puntuations_one_hot, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars_puntuations_one_hot, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences, test_chars_puntuations_one_hot, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
hub_embedding = hub.KerasLayer(
    "https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",
    trainable=False,
    name="universal_sentence_encoder",
)


token_inputs = tf.keras.layers.Input(shape=[], dtype='string', name='token_inputs')
token_embedding = hub_embedding(token_inputs)
token_x = tf.keras.layers.Dense(512, activation='relu')(token_embedding)
token_x = tf.keras.layers.Dense(256, activation='relu')(token_x)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_x)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(20,26,), dtype=tf.float32, name='char_puntuation_inputs')
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(24))(char_inputs)
char_outputs = tf.keras.layers.Dense(128, activation='relu')(char_bi_lstm)
char_model = tf.keras.Model(char_inputs, char_outputs)

line_number_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='line_number_inputs')
line_number_outputs = tf.keras.layers.Dense(32, activation='relu')(line_number_inputs)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(1,), name='total_lines_inputs')
total_lines_outputs = tf.keras.layers.Dense(32, activation='relu')(total_lines_inputs)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)
drop_out = tf.keras.layers.Dropout(0.5)(drop_out)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(final_concatenate)

model_m_35 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_35.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_35_history = model_m_35.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_35"),
)

model_m_35.load_weights("skim_lit/checkpoint/model_m_35/checkpoint.ckpt")

model_m_35_val_preds = tf.argmax(model_m_35.predict(val_dataset), axis=1)
model_m_35_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_35_val_preds)
print(model_m_35_val_results)

plot_loss_curves(model_m_35_history)

model_m_35_preds = tf.argmax(model_m_35.predict(test_dataset), axis=1)
model_m_35_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_35_preds)
model_m_35_results

## Model m 37

In [ ]:
import string

train_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in train_sentences]
val_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in val_sentences]
test_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in test_sentences]

sentence_lengths = [len(sentence.split(' ')) for sentence in train_chars_puntuations]
# sentence_lengths
seq_char_punctuation_len = int(np.percentile(sentence_lengths, 95) / 4) * 4

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    output_sequence_length=seq_char_punctuation_len,
    standardize='lower',
)

char_vectorizer.adapt(train_chars_puntuations)

train_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(train_chars_puntuations)))
val_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(val_chars_puntuations)))
test_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(test_chars_puntuations)))

one_hot_matrix = np.eye(len(char_vectorizer.get_vocabulary()))

train_chars_puntuations_one_hot = one_hot_matrix[train_chars_puntuations_vectorized][:, :, 2:]
val_chars_puntuations_one_hot = one_hot_matrix[val_chars_puntuations_vectorized][:, :, 2:]
test_chars_puntuations_one_hot = one_hot_matrix[test_chars_puntuations_vectorized][:, :, 2:]
train_chars_puntuations_one_hot.shape, val_chars_puntuations_one_hot.shape, test_chars_puntuations_one_hot.shape

In [ ]:
def tf_count(t, val):
    elements_equal_to_value = tf.equal(t, val)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    return count
text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=50)
text_vectorizer.adapt(train_sentences)
vocab = text_vectorizer.get_vocabulary()[2:]

words = tf.reshape(tf.strings.split(tf.strings.regex_replace(train_sentences, r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', ""), ' '), shape=[-1])
counts = {word: tf_count(words, word).numpy() for word in vocab}
counts_df = pd.DataFrame(counts.items(), columns=["word", "count"])
# counts_df = counts_df.sort_values("count", ascending=False)
max_tokens = counts_df[counts_df['count'] >= 5].shape[0]
max_tokens

In [ ]:
train_line_numbers_one_hot = train_data_10["line_number"].to_numpy()
val_line_numbers_one_hot = val_data_10["line_number"].to_numpy()
test_line_numbers_one_hot = test_df_100["line_number"].to_numpy()

train_total_lines_one_hot = train_data_10["total_lines"].to_numpy()
val_total_lines_one_hot = val_data_10["total_lines"].to_numpy()
test_total_lines_one_hot = test_df_100["total_lines"].to_numpy()

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars_puntuations_one_hot, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars_puntuations_one_hot, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences, test_chars_puntuations_one_hot, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=50,
)

text_vectorizer.adapt(train_sentences)

embedding = tf.keras.layers.Embedding(
    input_dim=len(text_vectorizer.get_vocabulary()),
    output_dim=1024,
    mask_zero=True,
)

token_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
token_vectorization = text_vectorizer(token_inputs)
token_embedding = embedding(token_vectorization)
token_merge = tf.keras.layers.Flatten()(token_embedding)
token_x = tf.keras.layers.Dense(512, activation='relu')(token_merge)
token_x = tf.keras.layers.Dense(256, activation='relu')(token_x)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_x)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(20,26,), dtype=tf.float32, name='char_puntuation_inputs')
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(char_inputs)
char_x = tf.keras.layers.Dense(256, activation='relu')(char_bi_lstm)
char_outputs = tf.keras.layers.Dense(128, activation='relu')(char_x)
char_model = tf.keras.Model(char_inputs, char_outputs)

line_number_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='line_number_inputs')
line_number_outputs = tf.keras.layers.Dense(32, activation='relu')(line_number_inputs)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(1,), name='total_lines_inputs')
total_lines_outputs = tf.keras.layers.Dense(32, activation='relu')(total_lines_inputs)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)
drop_out = tf.keras.layers.Dropout(0.5)(drop_out)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(final_concatenate)

model_m_37 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_37.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_37_history = model_m_37.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_37"),
)

model_m_37.load_weights("skim_lit/checkpoint/model_m_37/checkpoint.ckpt")

model_m_37_val_preds = tf.argmax(model_m_37.predict(val_dataset), axis=1)
model_m_37_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_37_val_preds)
print(model_m_37_val_results)

plot_loss_curves(model_m_37_history)

model_m_37_preds = tf.argmax(model_m_37.predict(test_dataset), axis=1)
model_m_37_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_37_preds)
model_m_37_results

## Model m 40

In [ ]:
import string

train_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in train_sentences]
val_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in val_sentences]
test_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in test_sentences]

sentence_lengths = [len(sentence.split(' ')) for sentence in train_chars_puntuations]
# sentence_lengths
seq_char_punctuation_len = int(np.percentile(sentence_lengths, 95) / 4) * 4

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    output_sequence_length=seq_char_punctuation_len,
    standardize='lower',
)

char_vectorizer.adapt(train_chars_puntuations)

train_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(train_chars_puntuations)))
val_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(val_chars_puntuations)))
test_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(test_chars_puntuations)))

one_hot_matrix = np.eye(len(char_vectorizer.get_vocabulary()))

train_chars_puntuations_one_hot = one_hot_matrix[train_chars_puntuations_vectorized][:, :, 2:]
val_chars_puntuations_one_hot = one_hot_matrix[val_chars_puntuations_vectorized][:, :, 2:]
test_chars_puntuations_one_hot = one_hot_matrix[test_chars_puntuations_vectorized][:, :, 2:]
train_chars_puntuations_one_hot.shape, val_chars_puntuations_one_hot.shape, test_chars_puntuations_one_hot.shape

In [ ]:
train_line_numbers_one_hot = train_data_10["line_number"].to_numpy()
val_line_numbers_one_hot = val_data_10["line_number"].to_numpy()
test_line_numbers_one_hot = test_df_100["line_number"].to_numpy()

train_total_lines_one_hot = train_data_10["total_lines"].to_numpy()
val_total_lines_one_hot = val_data_10["total_lines"].to_numpy()
test_total_lines_one_hot = test_df_100["total_lines"].to_numpy()

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars_puntuations_one_hot, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars_puntuations_one_hot, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences, test_chars_puntuations_one_hot, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
hub_embedding = hub.KerasLayer(
    "https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",
    trainable=False,
    name="universal_sentence_encoder",
)


token_inputs = tf.keras.layers.Input(shape=[], dtype='string', name='token_inputs')
token_embedding = hub_embedding(token_inputs)
token_x = tf.keras.layers.Dense(512, activation='relu')(token_embedding)
token_x = tf.keras.layers.Dense(256, activation='relu')(token_x)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_x)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(20,26,), dtype=tf.float32, name='char_puntuation_inputs')
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(24))(char_inputs)
char_outputs = tf.keras.layers.Dense(128, activation='relu')(char_bi_lstm)
char_model = tf.keras.Model(char_inputs, char_outputs)

line_number_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='line_number_inputs')
line_number_outputs = tf.keras.layers.Dense(32, activation='relu')(line_number_inputs)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(1,), name='total_lines_inputs')
total_lines_outputs = tf.keras.layers.Dense(32, activation='relu')(total_lines_inputs)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)
drop_out = tf.keras.layers.Dropout(0.5)(drop_out)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

x = tf.keras.layers.Dense(256, activation='relu')(final_concatenate)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(x)

model_m_40 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_40.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_40_history = model_m_40.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_40"),
)

model_m_40.load_weights("skim_lit/checkpoint/model_m_40/checkpoint.ckpt")

model_m_40_val_preds = tf.argmax(model_m_40.predict(val_dataset), axis=1)
model_m_40_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_40_val_preds)
print(model_m_40_val_results)

plot_loss_curves(model_m_40_history)

model_m_40_preds = tf.argmax(model_m_40.predict(test_dataset), axis=1)
model_m_40_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_40_preds)
model_m_40_results

## Model m 43

In [ ]:
import string

train_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in train_sentences]
val_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in val_sentences]
test_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in test_sentences]

sentence_lengths = [len(sentence.split(' ')) for sentence in train_chars_puntuations]
# sentence_lengths
seq_char_punctuation_len = int(np.percentile(sentence_lengths, 95) / 4) * 4

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    output_sequence_length=seq_char_punctuation_len,
    standardize='lower',
)

char_vectorizer.adapt(train_chars_puntuations)

train_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(train_chars_puntuations)))
val_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(val_chars_puntuations)))
test_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(test_chars_puntuations)))

one_hot_matrix = np.eye(len(char_vectorizer.get_vocabulary()))

train_chars_puntuations_one_hot = one_hot_matrix[train_chars_puntuations_vectorized][:, :, 2:]
val_chars_puntuations_one_hot = one_hot_matrix[val_chars_puntuations_vectorized][:, :, 2:]
test_chars_puntuations_one_hot = one_hot_matrix[test_chars_puntuations_vectorized][:, :, 2:]
train_chars_puntuations_one_hot.shape, val_chars_puntuations_one_hot.shape, test_chars_puntuations_one_hot.shape

In [ ]:
def tf_count(t, val):
    elements_equal_to_value = tf.equal(t, val)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    return count
text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=50)
text_vectorizer.adapt(train_sentences)
vocab = text_vectorizer.get_vocabulary()[2:]

words = tf.reshape(tf.strings.split(tf.strings.regex_replace(train_sentences, r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', ""), ' '), shape=[-1])
counts = {word: tf_count(words, word).numpy() for word in vocab}
counts_df = pd.DataFrame(counts.items(), columns=["word", "count"])
# counts_df = counts_df.sort_values("count", ascending=False)
max_tokens = counts_df[counts_df['count'] >= 5].shape[0]
max_tokens

In [ ]:
train_line_numbers_one_hot = train_data_10["line_number"].to_numpy()
val_line_numbers_one_hot = val_data_10["line_number"].to_numpy()
test_line_numbers_one_hot = test_df_100["line_number"].to_numpy()

train_total_lines_one_hot = train_data_10["total_lines"].to_numpy()
val_total_lines_one_hot = val_data_10["total_lines"].to_numpy()
test_total_lines_one_hot = test_df_100["total_lines"].to_numpy()

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars_puntuations_one_hot, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars_puntuations_one_hot, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences, test_chars_puntuations_one_hot, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=50,
)

text_vectorizer.adapt(train_sentences)

embedding = tf.keras.layers.Embedding(
    input_dim=len(text_vectorizer.get_vocabulary()),
    output_dim=1024,
    mask_zero=True,
)

token_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
token_vectorization = text_vectorizer(token_inputs)
token_embedding = embedding(token_vectorization)
token_merge = tf.keras.layers.Flatten()(token_embedding)
token_x = tf.keras.layers.Dense(512, activation='relu')(token_merge)
token_x = tf.keras.layers.Dense(256, activation='relu')(token_x)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_x)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(20,26,), dtype=tf.float32, name='char_puntuation_inputs')
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(char_inputs)
char_x = tf.keras.layers.Dense(256, activation='relu')(char_bi_lstm)
char_outputs = tf.keras.layers.Dense(128, activation='relu')(char_x)
char_model = tf.keras.Model(char_inputs, char_outputs)

line_number_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='line_number_inputs')
line_number_x = tf.keras.layers.Dense(256, activation='relu')(line_number_inputs)
line_number_x = tf.keras.layers.Dense(128, activation='relu')(line_number_x)
line_number_outputs = tf.keras.layers.Dense(64, activation='relu')(line_number_x)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(1,), name='total_lines_inputs')
total_lines_x = tf.keras.layers.Dense(256, activation='relu')(total_lines_inputs)
total_lines_x = tf.keras.layers.Dense(128, activation='relu')(total_lines_x)
total_lines_outputs = tf.keras.layers.Dense(64, activation='relu')(total_lines_x)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)
drop_out = tf.keras.layers.Dropout(0.5)(drop_out)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

x = tf.keras.layers.Dense(256, activation='relu')(final_concatenate)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(x)

model_m_43 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_43.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_43_history = model_m_43.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_43"),
)

model_m_43.load_weights("skim_lit/checkpoint/model_m_43/checkpoint.ckpt")

model_m_43_val_preds = tf.argmax(model_m_43.predict(val_dataset), axis=1)
model_m_43_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_43_val_preds)
print(model_m_43_val_results)

plot_loss_curves(model_m_43_history)

model_m_43_preds = tf.argmax(model_m_43.predict(test_dataset), axis=1)
model_m_43_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_43_preds)
model_m_43_results

## Model m 45

In [ ]:
import string

train_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in train_sentences]
val_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in val_sentences]
test_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in test_sentences]

sentence_lengths = [len(sentence.split(' ')) for sentence in train_chars_puntuations]
# sentence_lengths
seq_char_punctuation_len = int(np.percentile(sentence_lengths, 95) / 4) * 4

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    output_sequence_length=seq_char_punctuation_len,
    standardize='lower',
)

char_vectorizer.adapt(train_chars_puntuations)

train_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(train_chars_puntuations)))
val_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(val_chars_puntuations)))
test_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(test_chars_puntuations)))

one_hot_matrix = np.eye(len(char_vectorizer.get_vocabulary()))

train_chars_puntuations_one_hot = one_hot_matrix[train_chars_puntuations_vectorized][:, :, 2:]
val_chars_puntuations_one_hot = one_hot_matrix[val_chars_puntuations_vectorized][:, :, 2:]
test_chars_puntuations_one_hot = one_hot_matrix[test_chars_puntuations_vectorized][:, :, 2:]
train_chars_puntuations_one_hot.shape, val_chars_puntuations_one_hot.shape, test_chars_puntuations_one_hot.shape

In [ ]:
def tf_count(t, val):
    elements_equal_to_value = tf.equal(t, val)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    return count
text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=50)
text_vectorizer.adapt(train_sentences)
vocab = text_vectorizer.get_vocabulary()[2:]

words = tf.reshape(tf.strings.split(tf.strings.regex_replace(train_sentences, r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', ""), ' '), shape=[-1])
counts = {word: tf_count(words, word).numpy() for word in vocab}
counts_df = pd.DataFrame(counts.items(), columns=["word", "count"])
# counts_df = counts_df.sort_values("count", ascending=False)
max_tokens = counts_df[counts_df['count'] >= 5].shape[0]
max_tokens

In [ ]:
train_line_numbers_one_hot = train_data_10["line_number"].to_numpy()
val_line_numbers_one_hot = val_data_10["line_number"].to_numpy()
test_line_numbers_one_hot = test_df_100["line_number"].to_numpy()

train_total_lines_one_hot = train_data_10["total_lines"].to_numpy()
val_total_lines_one_hot = val_data_10["total_lines"].to_numpy()
test_total_lines_one_hot = test_df_100["total_lines"].to_numpy()

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars_puntuations_one_hot, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars_puntuations_one_hot, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences, test_chars_puntuations_one_hot, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=50,
)

text_vectorizer.adapt(train_sentences)

embedding = tf.keras.layers.Embedding(
    input_dim=len(text_vectorizer.get_vocabulary()),
    output_dim=1024,
    mask_zero=True,
)

token_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
token_vectorization = text_vectorizer(token_inputs)
token_embedding = embedding(token_vectorization)
token_merge = tf.keras.layers.Flatten()(token_embedding)
token_x = tf.keras.layers.Dense(512, activation='relu')(token_merge)
token_x = tf.keras.layers.Dense(256, activation='relu')(token_x)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_x)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(20,26,), dtype=tf.float32, name='char_puntuation_inputs')
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(char_inputs)
char_x = tf.keras.layers.Dense(256, activation='relu')(char_bi_lstm)
char_outputs = tf.keras.layers.Dense(128, activation='relu')(char_x)
char_model = tf.keras.Model(char_inputs, char_outputs)

line_number_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='line_number_inputs')
line_number_outputs = tf.keras.layers.Dense(32, activation='relu')(line_number_inputs)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(1,), name='total_lines_inputs')
total_lines_outputs = tf.keras.layers.Dense(32, activation='relu')(total_lines_inputs)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(final_concatenate)

model_m_45 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_45.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_45_history = model_m_45.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_45"),
)

model_m_45.load_weights("skim_lit/checkpoint/model_m_45/checkpoint.ckpt")

model_m_45_val_preds = tf.argmax(model_m_45.predict(val_dataset), axis=1)
model_m_45_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_45_val_preds)
print(model_m_45_val_results)

plot_loss_curves(model_m_45_history)

model_m_45_preds = tf.argmax(model_m_45.predict(test_dataset), axis=1)
model_m_45_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_45_preds)
model_m_45_results

## Model m 51

In [ ]:
import string

train_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in train_sentences]
val_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in val_sentences]
test_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in test_sentences]

sentence_lengths = [len(sentence.split(' ')) for sentence in train_chars_puntuations]
# sentence_lengths
seq_char_punctuation_len = int(np.percentile(sentence_lengths, 95) / 4) * 4

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    output_sequence_length=seq_char_punctuation_len,
    standardize='lower',
)

char_vectorizer.adapt(train_chars_puntuations)

train_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(train_chars_puntuations)))
val_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(val_chars_puntuations)))
test_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(test_chars_puntuations)))

one_hot_matrix = np.eye(len(char_vectorizer.get_vocabulary()))

train_chars_puntuations_one_hot = one_hot_matrix[train_chars_puntuations_vectorized][:, :, 2:]
val_chars_puntuations_one_hot = one_hot_matrix[val_chars_puntuations_vectorized][:, :, 2:]
test_chars_puntuations_one_hot = one_hot_matrix[test_chars_puntuations_vectorized][:, :, 2:]
train_chars_puntuations_one_hot.shape, val_chars_puntuations_one_hot.shape, test_chars_puntuations_one_hot.shape

In [ ]:
def tf_count(t, val):
    elements_equal_to_value = tf.equal(t, val)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    return count
text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=50)
text_vectorizer.adapt(train_sentences)
vocab = text_vectorizer.get_vocabulary()[2:]

words = tf.reshape(tf.strings.split(tf.strings.regex_replace(train_sentences, r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', ""), ' '), shape=[-1])
counts = {word: tf_count(words, word).numpy() for word in vocab}
counts_df = pd.DataFrame(counts.items(), columns=["word", "count"])
# counts_df = counts_df.sort_values("count", ascending=False)
max_tokens = counts_df[counts_df['count'] >= 5].shape[0]
max_tokens

In [ ]:
train_line_numbers_one_hot = train_data_10["line_number"].to_numpy()
val_line_numbers_one_hot = val_data_10["line_number"].to_numpy()
test_line_numbers_one_hot = test_df_100["line_number"].to_numpy()

train_total_lines_one_hot = train_data_10["total_lines"].to_numpy()
val_total_lines_one_hot = val_data_10["total_lines"].to_numpy()
test_total_lines_one_hot = test_df_100["total_lines"].to_numpy()

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars_puntuations_one_hot, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars_puntuations_one_hot, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences, test_chars_puntuations_one_hot, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=50,
)

text_vectorizer.adapt(train_sentences)

embedding = tf.keras.layers.Embedding(
    input_dim=len(text_vectorizer.get_vocabulary()),
    output_dim=1024,
    mask_zero=True,
)

token_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
token_vectorization = text_vectorizer(token_inputs)
token_embedding = embedding(token_vectorization)
token_merge = tf.keras.layers.Flatten()(token_embedding)
token_x = tf.keras.layers.Dense(512, activation='relu')(token_merge)
token_x = tf.keras.layers.Dense(256, activation='relu')(token_x)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_x)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(20,26,), dtype=tf.float32, name='char_puntuation_inputs')
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(char_inputs)
char_x = tf.keras.layers.Dense(256, activation='relu')(char_bi_lstm)
char_outputs = tf.keras.layers.Dense(128, activation='relu')(char_x)
char_model = tf.keras.Model(char_inputs, char_outputs)

line_number_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='line_number_inputs')
line_number_outputs = tf.keras.layers.Dense(32, activation='relu')(line_number_inputs)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(1,), name='total_lines_inputs')
total_lines_outputs = tf.keras.layers.Dense(32, activation='relu')(total_lines_inputs)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(final_concatenate)

model_m_51 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_51.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_51_history = model_m_51.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_51"),
)

model_m_51.load_weights("skim_lit/checkpoint/model_m_51/checkpoint.ckpt")

model_m_51_val_preds = tf.argmax(model_m_51.predict(val_dataset), axis=1)
model_m_51_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_51_val_preds)
print(model_m_51_val_results)

plot_loss_curves(model_m_51_history)

model_m_51_preds = tf.argmax(model_m_51.predict(test_dataset), axis=1)
model_m_51_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_51_preds)
model_m_51_results

## Model m 54

In [ ]:
import string

train_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in train_sentences]
val_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in val_sentences]
test_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in test_sentences]

sentence_lengths = [len(sentence.split(' ')) for sentence in train_chars_puntuations]
# sentence_lengths
seq_char_punctuation_len = int(np.percentile(sentence_lengths, 95) / 4) * 4

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    output_sequence_length=seq_char_punctuation_len,
    standardize='lower',
)

char_vectorizer.adapt(train_chars_puntuations)

train_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(train_chars_puntuations)))
val_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(val_chars_puntuations)))
test_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(test_chars_puntuations)))

one_hot_matrix = np.eye(len(char_vectorizer.get_vocabulary()))

train_chars_puntuations_one_hot = one_hot_matrix[train_chars_puntuations_vectorized][:, :, 2:]
val_chars_puntuations_one_hot = one_hot_matrix[val_chars_puntuations_vectorized][:, :, 2:]
test_chars_puntuations_one_hot = one_hot_matrix[test_chars_puntuations_vectorized][:, :, 2:]
train_chars_puntuations_one_hot.shape, val_chars_puntuations_one_hot.shape, test_chars_puntuations_one_hot.shape

In [ ]:
def tf_count(t, val):
    elements_equal_to_value = tf.equal(t, val)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    return count
text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=50)
text_vectorizer.adapt(train_sentences)
vocab = text_vectorizer.get_vocabulary()[2:]

words = tf.reshape(tf.strings.split(tf.strings.regex_replace(train_sentences, r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', ""), ' '), shape=[-1])
counts = {word: tf_count(words, word).numpy() for word in vocab}
counts_df = pd.DataFrame(counts.items(), columns=["word", "count"])
# counts_df = counts_df.sort_values("count", ascending=False)
max_tokens = counts_df[counts_df['count'] >= 5].shape[0]
max_tokens

In [ ]:
train_line_numbers_one_hot = train_data_10["line_number"].to_numpy()
val_line_numbers_one_hot = val_data_10["line_number"].to_numpy()
test_line_numbers_one_hot = test_df_100["line_number"].to_numpy()

train_total_lines_one_hot = train_data_10["total_lines"].to_numpy()
val_total_lines_one_hot = val_data_10["total_lines"].to_numpy()
test_total_lines_one_hot = test_df_100["total_lines"].to_numpy()

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars_puntuations_one_hot, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars_puntuations_one_hot, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences, test_chars_puntuations_one_hot, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=50,
)

text_vectorizer.adapt(train_sentences)

embedding = tf.keras.layers.Embedding(
    input_dim=len(text_vectorizer.get_vocabulary()),
    output_dim=1024,
    mask_zero=True,
)

token_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
token_vectorization = text_vectorizer(token_inputs)
token_embedding = embedding(token_vectorization)
token_merge = tf.keras.layers.Flatten()(token_embedding)
token_x = tf.keras.layers.Dense(4096, activation='relu')(token_merge)
token_x = tf.keras.layers.Dense(1024, activation='relu')(token_merge)
token_x = tf.keras.layers.Dense(512, activation='relu')(token_merge)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_x)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(20,26,), dtype=tf.float32, name='char_puntuation_inputs')
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(char_inputs)
char_x = tf.keras.layers.Dense(256, activation='relu')(char_bi_lstm)
char_outputs = tf.keras.layers.Dense(128, activation='relu')(char_x)
char_model = tf.keras.Model(char_inputs, char_outputs)

line_number_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='line_number_inputs')
line_number_outputs = tf.keras.layers.Dense(32, activation='relu')(line_number_inputs)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(1,), name='total_lines_inputs')
total_lines_outputs = tf.keras.layers.Dense(32, activation='relu')(total_lines_inputs)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)
drop_out = tf.keras.layers.Dropout(0.5)(drop_out)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(final_concatenate)

model_m_54 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_54.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_54_history = model_m_54.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_54"),
)

model_m_54.load_weights("skim_lit/checkpoint/model_m_54/checkpoint.ckpt")

model_m_54_val_preds = tf.argmax(model_m_54.predict(val_dataset), axis=1)
model_m_54_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_54_val_preds)
print(model_m_54_val_results)

plot_loss_curves(model_m_54_history)

model_m_54_preds = tf.argmax(model_m_54.predict(test_dataset), axis=1)
model_m_54_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_54_preds)
model_m_54_results

## Model m 55

In [ ]:
import string

train_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in train_sentences]
val_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in val_sentences]
test_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in test_sentences]

sentence_lengths = [len(sentence.split(' ')) for sentence in train_chars_puntuations]
# sentence_lengths
seq_char_punctuation_len = int(np.percentile(sentence_lengths, 95) / 4) * 4

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    output_sequence_length=seq_char_punctuation_len,
    standardize='lower',
)

char_vectorizer.adapt(train_chars_puntuations)

train_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(train_chars_puntuations)))
val_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(val_chars_puntuations)))
test_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(test_chars_puntuations)))

one_hot_matrix = np.eye(len(char_vectorizer.get_vocabulary()))

train_chars_puntuations_one_hot = one_hot_matrix[train_chars_puntuations_vectorized][:, :, 2:]
val_chars_puntuations_one_hot = one_hot_matrix[val_chars_puntuations_vectorized][:, :, 2:]
test_chars_puntuations_one_hot = one_hot_matrix[test_chars_puntuations_vectorized][:, :, 2:]
train_chars_puntuations_one_hot.shape, val_chars_puntuations_one_hot.shape, test_chars_puntuations_one_hot.shape

In [ ]:
def tf_count(t, val):
    elements_equal_to_value = tf.equal(t, val)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    return count
text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=50)
text_vectorizer.adapt(train_sentences)
vocab = text_vectorizer.get_vocabulary()[2:]

words = tf.reshape(tf.strings.split(tf.strings.regex_replace(train_sentences, r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', ""), ' '), shape=[-1])
counts = {word: tf_count(words, word).numpy() for word in vocab}
counts_df = pd.DataFrame(counts.items(), columns=["word", "count"])
# counts_df = counts_df.sort_values("count", ascending=False)
max_tokens = counts_df[counts_df['count'] >= 5].shape[0]
max_tokens

In [ ]:
train_line_numbers_one_hot = train_data_10["line_number"].to_numpy()
val_line_numbers_one_hot = val_data_10["line_number"].to_numpy()
test_line_numbers_one_hot = test_df_100["line_number"].to_numpy()

train_total_lines_one_hot = train_data_10["total_lines"].to_numpy()
val_total_lines_one_hot = val_data_10["total_lines"].to_numpy()
test_total_lines_one_hot = test_df_100["total_lines"].to_numpy()

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars_puntuations_one_hot, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars_puntuations_one_hot, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences, test_chars_puntuations_one_hot, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=50,
)

text_vectorizer.adapt(train_sentences)

embedding = tf.keras.layers.Embedding(
    input_dim=len(text_vectorizer.get_vocabulary()),
    output_dim=2048,
    mask_zero=True,
)

token_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
token_vectorization = text_vectorizer(token_inputs)
token_embedding = embedding(token_vectorization)
token_merge = tf.keras.layers.Flatten()(token_embedding)
token_x = tf.keras.layers.Dense(512, activation='relu')(token_merge)
token_x = tf.keras.layers.Dense(256, activation='relu')(token_x)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_x)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(20,26,), dtype=tf.float32, name='char_puntuation_inputs')
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(char_inputs)
char_x = tf.keras.layers.Dense(256, activation='relu')(char_bi_lstm)
char_outputs = tf.keras.layers.Dense(128, activation='relu')(char_x)
char_model = tf.keras.Model(char_inputs, char_outputs)

line_number_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='line_number_inputs')
line_number_outputs = tf.keras.layers.Dense(32, activation='relu')(line_number_inputs)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(1,), name='total_lines_inputs')
total_lines_outputs = tf.keras.layers.Dense(32, activation='relu')(total_lines_inputs)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)
drop_out = tf.keras.layers.Dropout(0.5)(drop_out)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(final_concatenate)

model_m_55 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_55.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_55_history = model_m_55.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_55"),
)

model_m_55.load_weights("skim_lit/checkpoint/model_m_55/checkpoint.ckpt")

model_m_55_val_preds = tf.argmax(model_m_55.predict(val_dataset), axis=1)
model_m_55_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_55_val_preds)
print(model_m_55_val_results)

plot_loss_curves(model_m_55_history)

model_m_55_preds = tf.argmax(model_m_55.predict(test_dataset), axis=1)
model_m_55_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_55_preds)
model_m_55_results

## Model m 56

In [ ]:
import string

train_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in train_sentences]
val_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in val_sentences]
test_chars_puntuations = [' '.join([e for e in list(sentence) if e not in string.ascii_lowercase + ' ']) for sentence in test_sentences]

sentence_lengths = [len(sentence.split(' ')) for sentence in train_chars_puntuations]
# sentence_lengths
seq_char_punctuation_len = int(np.percentile(sentence_lengths, 95) / 4) * 4

char_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    output_sequence_length=seq_char_punctuation_len,
    standardize='lower',
)

char_vectorizer.adapt(train_chars_puntuations)

train_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(train_chars_puntuations)))
val_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(val_chars_puntuations)))
test_chars_puntuations_vectorized = np.array(char_vectorizer(np.array(test_chars_puntuations)))

one_hot_matrix = np.eye(len(char_vectorizer.get_vocabulary()))

train_chars_puntuations_one_hot = one_hot_matrix[train_chars_puntuations_vectorized][:, :, 2:]
val_chars_puntuations_one_hot = one_hot_matrix[val_chars_puntuations_vectorized][:, :, 2:]
test_chars_puntuations_one_hot = one_hot_matrix[test_chars_puntuations_vectorized][:, :, 2:]
train_chars_puntuations_one_hot.shape, val_chars_puntuations_one_hot.shape, test_chars_puntuations_one_hot.shape

In [ ]:
def tf_count(t, val):
    elements_equal_to_value = tf.equal(t, val)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    return count
text_vectorizer = tf.keras.layers.TextVectorization(output_sequence_length=50)
text_vectorizer.adapt(train_sentences)
vocab = text_vectorizer.get_vocabulary()[2:]

words = tf.reshape(tf.strings.split(tf.strings.regex_replace(train_sentences, r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', ""), ' '), shape=[-1])
counts = {word: tf_count(words, word).numpy() for word in vocab}
counts_df = pd.DataFrame(counts.items(), columns=["word", "count"])
# counts_df = counts_df.sort_values("count", ascending=False)
max_tokens = counts_df[counts_df['count'] >= 5].shape[0]
max_tokens

In [ ]:
train_line_numbers_one_hot = train_data_10["line_number"].to_numpy()
val_line_numbers_one_hot = val_data_10["line_number"].to_numpy()
test_line_numbers_one_hot = test_df_100["line_number"].to_numpy()

train_total_lines_one_hot = train_data_10["total_lines"].to_numpy()
val_total_lines_one_hot = val_data_10["total_lines"].to_numpy()
test_total_lines_one_hot = test_df_100["total_lines"].to_numpy()

train_line_numbers_one_hot.shape, train_total_lines_one_hot.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=max_tokens)
train_sentences_vectorized = vectorizer.fit_transform(train_sentences).toarray()
val_sentences_vectorized = vectorizer.transform(val_sentences).toarray()
test_sentences_vectorized = vectorizer.transform(test_sentences).toarray()


In [ ]:
train_word_char_pos_data = tf.data.Dataset.from_tensor_slices((train_sentences_vectorized, train_chars_puntuations_one_hot, train_line_numbers_one_hot, train_total_lines_one_hot))
train_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_dataset = tf.data.Dataset.zip((train_word_char_pos_data, train_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

val_word_char_pos_data = tf.data.Dataset.from_tensor_slices((val_sentences_vectorized, val_chars_puntuations_one_hot, val_line_numbers_one_hot, val_total_lines_one_hot))
val_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_dataset = tf.data.Dataset.zip((val_word_char_pos_data, val_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

test_word_char_pos_data = tf.data.Dataset.from_tensor_slices((test_sentences_vectorized, test_chars_puntuations_one_hot, test_line_numbers_one_hot, test_total_lines_one_hot))
test_word_char_pos_labels = tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_dataset = tf.data.Dataset.zip((test_word_char_pos_data, test_word_char_pos_labels)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:


token_inputs = tf.keras.layers.Input(shape=(max_tokens,), dtype=tf.float32)
token_x = tf.keras.layers.Dense(512, activation='relu')(token_inputs)
token_x = tf.keras.layers.Dense(256, activation='relu')(token_x)
token_outputs = tf.keras.layers.Dense(128, activation='relu')(token_x)
token_model = tf.keras.Model(token_inputs, token_outputs)

char_inputs = tf.keras.layers.Input(shape=(20,26,), dtype=tf.float32, name='char_puntuation_inputs')
char_bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(char_inputs)
char_x = tf.keras.layers.Dense(256, activation='relu')(char_bi_lstm)
char_outputs = tf.keras.layers.Dense(128, activation='relu')(char_x)
char_model = tf.keras.Model(char_inputs, char_outputs)

line_number_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='line_number_inputs')
line_number_outputs = tf.keras.layers.Dense(32, activation='relu')(line_number_inputs)
line_number_model = tf.keras.Model(line_number_inputs, line_number_outputs)

total_lines_inputs = tf.keras.layers.Input(shape=(1,), name='total_lines_inputs')
total_lines_outputs = tf.keras.layers.Dense(32, activation='relu')(total_lines_inputs)
total_lines_model = tf.keras.Model(total_lines_inputs, total_lines_outputs)

token_char_concatenate = tf.keras.layers.Concatenate(name="token_char_concatenate")([token_model.output, char_model.output])

drop_out = tf.keras.layers.Dense(256, activation='relu')(token_char_concatenate)
drop_out = tf.keras.layers.Dropout(0.5)(drop_out)

final_concatenate = tf.keras.layers.Concatenate(name='final_concatenate')([drop_out, line_number_model.output, total_lines_model.output])

outputs = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(final_concatenate)

model_m_56 = tf.keras.Model([token_model.input, char_model.input, line_number_model.input, total_lines_model.input], outputs)

model_m_56.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(),
    metrics=["accuracy"],
)

model_m_56_history = model_m_56.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=int(len(train_dataset)),
    validation_data=val_dataset,
    validation_steps=int(len(val_dataset)),
    callbacks=get_callbacks("model_m_56"),
)

model_m_56.load_weights("skim_lit/checkpoint/model_m_56/checkpoint.ckpt")

model_m_56_val_preds = tf.argmax(model_m_56.predict(val_dataset), axis=1)
model_m_56_val_results = calculate_results(tf.argmax(val_labels_one_hot, axis=1), model_m_56_val_preds)
print(model_m_56_val_results)

plot_loss_curves(model_m_56_history)

model_m_56_preds = tf.argmax(model_m_56.predict(test_dataset), axis=1)
model_m_56_results = calculate_results(tf.argmax(test_labels_one_hot, axis=1), model_m_56_preds)
model_m_56_results

## Summary and Results of all models

In [ ]:
all_val_results = pd.DataFrame({
    "model_m_0": model_m_0_val_results,
    "model_m_1": model_m_1_val_results,
    "model_m_5": model_m_5_val_results,
    "model_m_23": model_m_23_val_results,
    "model_m_25": model_m_25_val_results,
    "model_m_32": model_m_32_val_results,
    "model_m_35": model_m_35_val_results,
    "model_m_37": model_m_37_val_results,
    "model_m_40": model_m_40_val_results,
    "model_m_43": model_m_43_val_results,
    "model_m_45": model_m_45_val_results,
    "model_m_51": model_m_51_val_results,
    "model_m_54": model_m_54_val_results,
    "model_m_55": model_m_55_val_results,
    "model_m_56": model_m_56_val_results,
})

all_val_results = all_val_results.T
all_val_results

In [ ]:
all_val_results.plot(kind="bar", figsize=(10, 7)).legend(bbox_to_anchor=(1.0, 1.0))

In [ ]:
all_val_results["accuracy"].plot(kind="bar", figsize=(10, 7)).legend(bbox_to_anchor=(1.0, 1.0))

In [ ]:
all_val_results['f1'].plot(kind="bar", figsize=(10, 7)).legend(bbox_to_anchor=(1.0, 1.0))

In [ ]:
all_results = pd.DataFrame({
    "model_m_0": model_m_0_results,
    "model_m_1": model_m_1_results,
    "model_m_5": model_m_5_results,
    "model_m_23": model_m_23_results,
    "model_m_25": model_m_25_results,
    "model_m_32": model_m_32_results,
    "model_m_35": model_m_35_results,
    "model_m_37": model_m_37_results,
    "model_m_40": model_m_40_results,
    "model_m_43": model_m_43_results,
    "model_m_45": model_m_45_results,
    "model_m_51": model_m_51_results,
    "model_m_54": model_m_54_results,
    "model_m_55": model_m_55_results,
    "model_m_56": model_m_56_results,
})

all_results = all_results.T
all_results

In [ ]:
all_results.plot(kind="bar", figsize=(10, 7)).legend(bbox_to_anchor=(1.0, 1.0))

In [ ]:
all_results["accuracy"].plot(kind="bar", figsize=(10, 7)).legend(bbox_to_anchor=(1.0, 1.0))

In [ ]:
all_results['f1'].plot(kind="bar", figsize=(10, 7)).legend(bbox_to_anchor=(1.0, 1.0))

In [ ]:
from datetime import datetime
print("Notebook last run at:")
print(datetime.now())